In [ ]:
import os
import pandas as pd
import numpy as np
from pulp import (LpProblem, LpVariable, LpBinary, LpMinimize, lpSum, LpStatus, value)
os.chdir(r"C:\Users\Anand\Documents")
os.getcwd()

"C:\\Users\\Anand\\Documents\\Georgia Tech Master's in Analytics\\Fall 2025\\ISYE 6501 - Intro to Analytics Modeling\\Week 11 - Optimization & Advanced Models\\HW"

#### Part 1

In [3]:
df = pd.read_excel("diet.xls").fillna(0)
foods_df = df.iloc[:64].copy()

food_col = "Foods"
price_col = "Price/ Serving"
nutr_cols = ["Calories","Cholesterol mg","Total_Fat g","Sodium mg", "Carbohydrates g",
             "Dietary_Fiber g","Protein g", "Vit_A IU","Vit_C IU","Calcium mg","Iron mg"]

# Parameters: c_i, a_ij, m_j, M_j
# Cost c_i
foods_df["Price"] = (foods_df[price_col].astype(str).str.replace(r"[\$,]", "", regex = True).astype(float))

# min / max
nutr_min = {
    "Calories":1500,"Cholesterol mg":30,"Total_Fat g":20,"Sodium mg":800,
    "Carbohydrates g":130,"Dietary_Fiber g":125,"Protein g":60,
    "Vit_A IU":1000,"Vit_C IU":400,"Calcium mg":700,"Iron mg":10
}
nutr_max = {
    "Calories":2500,"Cholesterol mg":240,"Total_Fat g":70,"Sodium mg":2000,
    "Carbohydrates g":450,"Dietary_Fiber g":250,"Protein g":100,
    "Vit_A IU":10000,"Vit_C IU":5000,"Calcium mg":1500,"Iron mg":40
}

# index sets
foods = foods_df[food_col].astype(str).tolist() #F
price = dict(zip(foods, foods_df["Price"]))     #c_i
A = {n: dict(zip(foods, foods_df[n])) for n in nutr_cols}   # a_ij

# Decision Variables x_i >= 0
model = LpProblem("Diet_Part1_MinCost", LpMinimize)
x = {f: LpVariable(f"x_{f.replace(' ', '_')}", lowBound = 0) for f in foods}

# Constraints: for each nutrient j
for n in nutr_cols:
    # minimum constraint
    model += lpSum(A[n][f]*x[f] for f in foods) >= nutr_min[n], f"Min_{n}"
    # maximum constraint
    model += lpSum(A[n][f]*x[f] for f in foods) <= nutr_max[n], f"Max_{n}"

# Objective: minimize sum(C_i*x_i)
model += lpSum(price[f]*x[f] for f in foods), "Total_Cost"


#### Part 1 Results

In [4]:
model.solve()
print("Status:", LpStatus[model.status])
print("Optimal Diet Cost = $", round(value(model.objective), 2), "\n")

result = (
    pd.DataFrame({
        "Food": foods,
        "Servings": [x[f].value() for f in foods],
        "Cost": [price[f]*(x[f].value() or 0) for f in foods]
    })
    .query("Servings > 0")
    .sort_values("Servings", ascending = False)
          )
print(result.to_string(index=False))

Status: Optimal
Optimal Diet Cost = $ 4.34 

               Food  Servings     Cost
Lettuce,Iceberg,Raw 63.988506 1.279770
        Celery, Raw 52.643710 2.105748
 Popcorn,Air-Popped 13.869322 0.554773
            Oranges  2.292939 0.343941
    Frozen Broccoli  0.259607 0.041537
       Poached Eggs  0.141844 0.011348


#### Part 2

In [5]:

model2 = LpProblem("Diet_Part2_MinCost_with_Constraints", LpMinimize)

# Decision Variables
x = {f: LpVariable(f"x_{f.replace(' ', '_')}", lowBound = 0) for f in foods}
y = {f: LpVariable(f"y_{f.replace(' ', '_')}", cat=LpBinary) for f in foods}

# Nutrition Constraints
for n in nutr_cols:
    # minimum constraint
    model2 += lpSum(A[n][f]*x[f] for f in foods) >= nutr_min[n], f"Min_{n}"
    # maximum constraint
    model2 += lpSum(A[n][f]*x[f] for f in foods) <= nutr_max[n], f"Max_{n}"

# (a) linking: 0.1 =< x_i =< M y_i
M = 100
for f in foods:
    model2 += x[f] >= 0.1*y[f], f"MinServ_if_Selected_{f}"
    model2 += x[f] <= M*y[f], f"MaxServ_if_Selected_{f}"

# (b) At most one of celery or broccoli
celery = [f for f in foods if "celery" in f.lower()]
broccoli = [f for f in foods if "broccoli" in f.lower()]
if celery or broccoli:
    model2 += lpSum(y[f] for f in celery + broccoli) <= 1, "Celery_or_Broccoli"

# (c) At least 3 protein items (meat/poultry/fish/eggs)
protein_keywords = [ "chicken","egg","beef","ham","pork","tuna","sardine","frankfurter",
                     "hamburger","hotdog","kielbasa","fish","bologna"]
protein_set = [f for f in foods if any(k in f.lower() for k in protein_keywords)]
model2 += lpSum(y[f] for f in protein_set) >= 3, "AtLeast3Proteins"

# Objective: minimize sum(C_i*x_i)
model2 += lpSum(price[f]*x[f] for f in foods), "Total_Cost"

#### Part 2 Results

In [6]:
model2.solve()
print("Status:", LpStatus[model2.status])
print("Optimal Diet Cost = $", round(value(model2.objective), 2), "\n")

result = (
    pd.DataFrame({
        "Food": foods,
        "Selected": [int(y[f].value() or 0) for f in foods],
        "Servings": [x[f].value() for f in foods],
        "Cost": [price[f]*(x[f].value() or 0) for f in foods]
    })
    .query("Selected == 1")
    .sort_values("Servings", ascending = False)
    .reset_index(drop=True)
          )
print(result[["Food", "Servings", "Cost"]].to_string(index=False))

Status: Optimal
Optimal Diet Cost = $ 4.51 

               Food  Servings     Cost
Lettuce,Iceberg,Raw 82.802586 1.656052
        Celery, Raw 42.399358 1.695974
 Popcorn,Air-Popped 13.223294 0.528932
            Oranges  3.077184 0.461578
      Peanut Butter  1.942972 0.136008
       Poached Eggs  0.100000 0.008000
       Kielbasa,Prk  0.100000 0.015000
     Scrambled Eggs  0.100000 0.011000


#### Optional Cholesterol Problem

In [7]:
df2 = pd.read_excel("diet_large.xls").fillna(0)

# Clean Column Names
df2.columns = (df2.columns
               .str.strip()
               .str.replace('\xa0', ' ', regex=False)
               .str.replace('\n', ' ', regex=False))

food_col = "Foods"
chol_col = "Cholesterol"
print(list(df2.columns))

['Foods', 'Protein', 'Carbohydrate, by difference', 'Energy1', 'Water', 'Energy2', 'Calcium, Ca', 'Iron, Fe', 'Magnesium, Mg', 'Phosphorus, P', 'Potassium, K', 'Sodium, Na', 'Zinc, Zn', 'Copper, Cu', 'Manganese, Mn', 'Selenium, Se', 'Vitamin A, RAE', 'Vitamin E (alpha-tocopherol)', 'Vitamin D', 'Vitamin C, total ascorbic acid', 'Thiamin', 'Riboflavin', 'Niacin', 'Pantothenic acid', 'Vitamin B-6', 'Folate, total', 'Vitamin B-12', 'Vitamin K (phylloquinone)', 'Cholesterol', 'Fatty acids, total trans', 'Fatty acids, total saturated']


In [13]:
nutr_headers = ['Protein', 'Carbohydrate, by difference', 'Energy1', 'Water', 'Energy2', 'Calcium, Ca', 'Iron, Fe', 'Magnesium, Mg', 
                'Phosphorus, P', 'Potassium, K', 'Sodium, Na', 'Zinc, Zn', 'Copper, Cu', 'Manganese, Mn', 'Selenium, Se',
                'Vitamin A, RAE', 'Vitamin E (alpha-tocopherol)', 'Vitamin D', 'Vitamin C, total ascorbic acid', 'Thiamin',
                'Riboflavin', 'Niacin', 'Pantothenic acid', 'Vitamin B-6', 'Folate, total', 'Vitamin B-12', 'Vitamin K (phylloquinone)']

min_values = [56, 130, 2400, 3700, 2400, 1000, 8, 270, 700, 4700, 1500, 11, 0.9, 2.3, 55,
              900, 15, 200, 90, 0.0012, 1.3, 16, 5, 1.3, 400, 2.4, 120]

max_values = [1000000, 1000000, 1000000, 1000000, 1000000, 2500, 45, 400, 4000, 1000000,
              2300, 40, 10, 11, 400, 3000, 1000, 2000, 2000, 1000000, 1000000, 35,
              1000000, 100, 1000, 1000000, 1000000]

# Create minimum and maximum rows as dictionary
nutr_min2 = dict(zip(nutr_headers, min_values))
nutr_max2 = dict(zip(nutr_headers, max_values))

# Cholesterol columns to numeric
for col in [chol_col] + nutr_headers:
    df2[col] = pd.to_numeric(df2[col], errors="coerce").fillna(0)

# Build LP
idxs = list(df2.index)
labels = df2[food_col].astype(str)

x = {i: LpVariable(f"x_{i}", lowBound=0) for i in idxs}

model3 = LpProblem("Diet_Large_Min_Chol", LpMinimize)

# Constraints
for n in nutr_headers:
    col = df2[n]
    # minimum constraint
    model3 += lpSum(col.at[i]*x[i] for i in idxs) >= nutr_min2[n], f"Min_{n}"
    # maximum constraint                    
    model3 += lpSum(col.at[i]*x[i] for i in idxs) <= nutr_max2[n], f"Max_{n}"   

# Objective
model3 += lpSum(float(df2.at[i, chol_col]) *x[i] for i in idxs), "Total_Cholesterol"

#### Print Cholesterol Results

In [15]:
model3.solve()
print("Status:", LpStatus[model3.status])
print("Total Cholesterol (mg):", round(value(model3.objective), 4))

res = pd.DataFrame({
    "Food": [labels.at[i] for i in idxs],
    "Servings": [x[i].value() for i in idxs],
    "Cholesterol (mg)": [df2.at[i, chol_col] * (x[i].value() or 0) for i in idxs]
})
res = res[res["Servings"] > 1e-6].sort_values("Servings", ascending=False)
print(res.head(30).to_string(index=False))

Status: Optimal
Total Cholesterol (mg): 0.0
                                                           Food    Servings  Cholesterol (mg)
                Water, bottled, non-carbonated, PEPSI, AQUAFINA 9993.727800               0.0
Infant formula, MEAD JOHNSON, ENFAMIL, NUTRAMIGEN LIPIL, with i    2.415677               0.0
                 Babyfood, vegetable, butternut squash and corn    1.884269               0.0
                                                 Arrowhead, raw    1.147862               0.0
Infant formula, MEAD JOHNSON,NEXT STEP PROSOBEE, prepared from     1.134890               0.0
                                  Cake, white, dry mix, regular    1.111002               0.0
                          Snacks, popcorn, oil-popped, unsalted    0.943146               0.0
                               Babyfood, fruit, peaches, junior    0.893873               0.0
               Babyfood, dinner, vegetables and turkey, toddler    0.871812               0.0
Infant formula, 